**Please open with Jupyter notebook.
Because COLAB can't access the webcam by OpenCV.**

In [ ]:
import cv2
import numpy as np
from keras.models import load_model

## We load the model
model=load_model("./model/face_mask_detector_model.h5")

results={0:'NO MASK', 1:'MASK'}
colors={0:(0,0,255), 1:(0,255,0)} #color

rect_size = 4

# Use camera 0
cap = cv2.VideoCapture(0) 

# We load the xml file
haarcascade = cv2.CascadeClassifier('./face_detector/haarcascade_frontalface_default.xml')

while True:    
    # image capture
    (rval, image) = cap.read()
    
    #Flip to act as a mirror
    image = cv2.flip(image,1,1) 
    
    # print our group name
    cv2.rectangle(image,
                  (0,0),
                  (640,35),
                  (245, 165, 66),
                  -1)
    cv2.putText(image,
                'NeuralBit, PUC',
                (10, 25),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.8,
                (255,255,255),
                2)
    
    # Resize the image to speed up detection
    rerect_size = cv2.resize(image,
                             (image.shape[1] // rect_size,
                              image.shape[0] // rect_size))
    
    # detect MultiScale / faces 
    faces = haarcascade.detectMultiScale(rerect_size)
    
    # Draw rectangles around each face
    for f in faces:
        #Scale the shapesize backup
        (x, y, w, h) = [v * rect_size for v in f] 
        #Scale the shapesize backup
        face_img = image[y:y+h, x:x+w]
        rerect_sized = cv2.resize(face_img,(150,150))
        normalized = rerect_sized/255.0
        reshaped = np.reshape(normalized,(1,150,150,3))
        reshaped = np.vstack([reshaped])
        
        # prediction
        result=model.predict(reshaped)
        accu = round(np.max(result)*100, 2)
        
        label = np.argmax(result,axis=1)[0] 
      
        cv2.rectangle(image, (x,y), (x+w,y+h), colors[label],1) #rectangle on face
        cv2.rectangle(image, (x,y-30), (x+w,y), colors[label],-1) #label_background
        cv2.putText(image, results[label]+' '+str(accu)+'%', (x+5, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.7,(255,255,255),1)
    
    # Show the image
    cv2.imshow('NeuralBit | Face Mask Detector',   image)
    # Press Q on keyboard to exit window
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Stop video
cap.release()
# Close all started windows
cv2.destroyAllWindows()